***GENERATED CODE FOR denialresolution PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '81', 'mean': '', 'stddev': '', 'min': '0234BE6B-6A09-4CFA-AD3F-64EFE02720ED', 'max': 'F79CCE6E-F180-4C2A-A977-4AA4F1C57B65', 'missing': '0', 'distinct': '73'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDate', 'transformation_label': 'Extract Date'}], 'feature': 'ReasonDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'ReasonDate'}, {'feature_label': 'ReasonDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('ReasonDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '223150.21', 'stddev': '1088546.67', 'min': '11042', 'max': 'J2405', 'missing': '0', 'distinct': '27'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '6.37', 'stddev': '12.12', 'min': '0000', 'max': 'GP', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier'}, {'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier2', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '0.73', 'stddev': '6.56', 'min': '0000', 'max': '59', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier2'}, {'feature_label': 'CptModifier2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'B35.4', 'max': 'Z01.818', 'missing': '0', 'distinct': '54'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'B35.0', 'max': 'Z99.2', 'missing': '0', 'distinct': '54'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'Cardiovascular', 'max': 'Supplies', 'missing': '0', 'distinct': '17'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Vascular Surgery', 'missing': '0', 'distinct': '13'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SpecialityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'Blue Cross Blue Sheild Of Florida', 'max': 'United Healthcare', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerCategory'}, {'feature_label': 'PrimaryPayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'BCBS', 'max': 'UHC', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerGroup'}, {'feature_label': 'PrimaryPayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerName', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'AARP 2ndary', 'max': 'Wellcare MCR HMO', 'missing': '0', 'distinct': '15'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerName'}, {'feature_label': 'PrimaryPayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerSubGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerSubGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'Medicaid', 'max': 'United Healthcare Medicare Advantage', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerSubGroup1'}, {'feature_label': 'PrimaryPayerSubGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerSubGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerSubGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerSubGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'AmeriGroup', 'max': 'United Healthcare Medicare Advantage', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerSubGroup2'}, {'feature_label': 'PrimaryPayerSubGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerSubGroup2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'Aetna US Healthcare', 'max': 'United Healthcare', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerCategory'}, {'feature_label': 'SecondPayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'UHC', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerGroup'}, {'feature_label': 'SecondPayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerName', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'AARP 2ndary', 'max': 'Wellcare Staywell', 'missing': '0', 'distinct': '17'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerName'}, {'feature_label': 'SecondPayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerSubGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerSubGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'Medicaid', 'max': 'T_90', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerSubGroup1'}, {'feature_label': 'SecondPayerSubGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerSubGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerSubGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerSubGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Wellcare/Staywell', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerSubGroup2'}, {'feature_label': 'SecondPayerSubGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerSubGroup2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceItemName', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceItemName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '223150.21', 'stddev': '1088546.67', 'min': '11042', 'max': 'J2405', 'missing': '0', 'distinct': '27'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceItemName'}, {'feature_label': 'ServiceItemName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceItemName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceItemCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceItemCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'CPTR OPHTH DX IMG POST SEGMT', 'max': 'Zinc paste band w >=3"<5"/yd', 'missing': '0', 'distinct': '24'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceItemCategory'}, {'feature_label': 'ServiceItemCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceItemCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceLocation', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceLocation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'Inpatient Hospital', 'max': 'Office', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceLocation'}, {'feature_label': 'ServiceLocation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceLocation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'reasonType', 'transformation_label': 'String Indexer'}], 'feature': 'reasonType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'Denial', 'max': 'Denial', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'reasonType'}, {'feature_label': 'reasonType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('reasonType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'}], 'feature': 'Reasoncode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '22.0', 'stddev': '0.0', 'min': '22', 'max': '22', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Reasoncode'}, {'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Reasoncode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'RemarkCode', 'transformation_label': 'String Indexer'}], 'feature': 'RemarkCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'N822', 'max': 'N822', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'RemarkCode'}, {'feature_label': 'RemarkCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('RemarkCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'COB', 'max': 'COB', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonCategory'}, {'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonDescription', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'This care may be covered by another payer per coordination of benefits.', 'max': 'This care may be covered by another payer per coordination of benefits.', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonDescription'}, {'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonDescription')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Remarkcode1', 'transformation_label': 'String Indexer'}], 'feature': 'Remarkcode1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '81', 'mean': '', 'stddev': '', 'min': 'CO22', 'max': 'PR22', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Remarkcode1'}, {'feature_label': 'Remarkcode1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Remarkcode1')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run denialresolutionHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_BOND_CLINIC', 'optionalDB': 'NextGen_BOND_CLINIC', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT\r\n\r\ncd.ChargeDetailID,\r\ntr.ReasonDate,\r\ncptc.CPTCode,\r\nCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\nCASE WHEN cd.CPTModifier2 IS NOT NULL THEN cd.CPTModifier2 ELSE '0000' END as CptModifier2,\r\ndc.DiagnosisCode as pdiagnosis1,\r\ndc2.DiagnosisCode as pdiagnosis2,\r\nFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\ncd.Amount,\r\nd.DepartmentCategory,\r\ns.SpecialityCategory,\r\n\r\nCASE WHEN py.PayerCategory IS NOT NULL THEN py.PayerCategory ELSE 'NOT AVAILABLE' END as PrimaryPayerCategory ,\r\nCASE WHEN py.PayerGroup IS NOT NULL THEN py.PayerGroup ELSE 'NOT AVAILABLE' END as PrimaryPayerGroup ,\r\nCASE WHEN py.PayerName IS NOT NULL THEN py.PayerName ELSE 'NOT AVAILABLE' END as PrimaryPayerName ,\r\nCASE WHEN py.PayerSubGroup1 IS NOT NULL THEN py.PayerSubGroup1 ELSE 'NOT AVAILABLE' END as PrimaryPayerSubGroup1 ,\r\nCASE WHEN py.PayerSubGroup2 IS NOT NULL THEN py.PayerSubGroup2 ELSE 'NOT AVAILABLE' END as PrimaryPayerSubGroup2 ,\r\n\r\nCASE WHEN py1.PayerCategory IS NOT NULL THEN py1.PayerCategory ELSE 'NOT AVAILABLE' END as SecondPayerCategory ,\r\nCASE WHEN py1.PayerGroup IS NOT NULL THEN py1.PayerGroup ELSE 'NOT AVAILABLE' END as SecondPayerGroup ,\r\nCASE WHEN py1.PayerName IS NOT NULL THEN py1.PayerName ELSE 'NOT AVAILABLE' END as SecondPayerName ,\r\nCASE WHEN py1.PayerSubGroup1 IS NOT NULL THEN py1.PayerSubGroup1 ELSE 'NOT AVAILABLE' END as SecondPayerSubGroup1 ,\r\nCASE WHEN py1.PayerSubGroup2 IS NOT NULL THEN py1.PayerSubGroup2 ELSE 'NOT AVAILABLE' END as SecondPayerSubGroup2 ,\r\n\r\nCASE WHEN si.ServiceItemName IS NOT NULL THEN si.ServiceItemName ELSE 'NOT AVAILABLE' END as ServiceItemName ,\r\nCASE WHEN si.ServiceItemCategory IS NOT NULL THEN si.ServiceItemCategory ELSE 'NOT AVAILABLE' END as ServiceItemCategory,\r\nCASE WHEN sl.ServiceLocation IS NOT NULL THEN sl.ServiceLocation ELSE 'NOT AVAILABLE' END as ServiceLocation,\r\n\r\n\r\n\r\ntrc.reasonType,\r\ntrc.TransactionReasonCodeID,\r\ntrc.Reasoncode,\r\nrc.SourceCode as RemarkCode,\r\ntrc.ReasonCategory,\r\ntrc.ReasonDescription,\r\ntr.SourceKey as Remarkcode1,\r\n\r\n\r\n\r\nCASE\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID IN ('3','9','12','47','50','56','73','84') THEN 1\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID IN ('4','44','54','62','66','67','76') THEN 2\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID IN ('5','45') THEN 3\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID IN ('6','11','46','48','52','58','70','71','75','78','28512') THEN 4\r\n\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 6 THEN 6\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 8 THEN 8\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 13 THEN 13\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 16 THEN 16\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 41 THEN 41\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 42 THEN 42\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 43 THEN 43\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 44 THEN 44\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 48 THEN 48\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 49 THEN 49\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 61 THEN 61\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 62 THEN 62\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 63 THEN 63\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 64 THEN 64\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 65 THEN 65\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 79 THEN 79\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 80 THEN 80\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 81 THEN 81\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 82 THEN 82\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 83 THEN 83\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 85 THEN 85\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 89 THEN 89\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 789 THEN 789\r\nWHEN t.transactiontypeid IN ('1','2') and tcr.TaskCompletionReasonID = 30899 THEN 30899\r\n\r\nELSE '0' END AS label_resolution,\r\nRR.Rank\r\n\r\nFROM\r\nChargeDetail cd\r\ninner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\ninner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID AND T.TransactionDate>=TR.ReasonDate\r\ninner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\ninner join ReasonCodeMapping rc on rc.RealCodeID=trc.TransactionReasonCodeID and (rc.SourceCode like'M%' or rc.SourceCode like'N%') and trc.ReasonType='Denial'\r\ninner join Payer py WITH (NOLOCK) on py.PayerID = t.PayerID\r\ninner join Transactiontype tt on tt.transactiontypeid = t.transactiontypeid\r\ninner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\nleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = t.PayerID\r\ninner join Bill b WITH (NOLOCK) on e.EncounterID = b.EncounterID and b.BillID = t.BillID\r\nleft outer join Task tsk WITH (NOLOCK) on e.encounterid = tsk.encounterid and e.facilityid = tsk.facilityid and tsk.createddate >= tr.reasondate\r\nleft outer join TaskCompletionReason tcr WITH (NOLOCK) on tsk.TaskCompletionReasonID = tcr.TaskCompletionReasonID\r\nleft outer join Taskstatus ts on tsk.TaskStatusID=ts.TaskStatusID\r\nleft outer join TaskType tskt on tsk.TaskTypeID=tskt.TaskTypeID\r\ninner join tmp_ResolutionRankings RR on tsk.TaskCompletionReasonID=RR.TaskCompletionReasonID\r\ninner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\ninner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\ninner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\ninner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\ninner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\ninner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\ninner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\nleft join Payer py1 WITH (NOLOCK) on py1.PayerID = cd.SecondaryPayerID\r\nleft join Payer py2 WITH (NOLOCK) on py2.PayerID = cd.TertiaryPayerID\r\ninner join ServiceItem si WITH (NOLOCK) on cd.ServiceItemID = si.ServiceItemID\r\nleft outer join ServiceLocation sl WITH (NOLOCK) on cd.ServiceLocationID = sl.ServiceLocationID\r\n\r\nWHERE cd.Amount >.01\r\nand trc.ReasonCode IN ('19','20','21','22','24','26','27','28','30','31','32','33','109','166','200')\r\nand trc.reasonType = 'Denial'\r\nand tr.ReasonDate between '2018-01-01' and '2020-12-31'\r\nand ts.TaskStatusCategory ='Completed'\r\nand tcr.TaskCompletionReasonID NOT IN ('99','100','101')\r\nand tcr.TaskCompletionReason !='NULL'\r\nand tsk.CreatedDate >= tr.ReasonDate\r\nand TransactionDate >= tsk.CompletedDate\r\nand tskt.TaskTypeCategory='AR'\r\nand RR.Rank = 1", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run denialresolutionHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "0234BE6B-6A09-4CFA-AD3F-64EFE02720ED", "max": "F79CCE6E-F180-4C2A-A977-4AA4F1C57B65", "missing": "0", "distinct": "73"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"feature_label": "ReasonDate", "transformation_label": "Extract Date"}], "feature": "ReasonDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "ReasonDate"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "223150.21", "stddev": "1088546.67", "min": "11042", "max": "J2405", "missing": "0", "distinct": "27"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "CptModifier", "transformation_label": "String Indexer"}], "feature": "CptModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "6.37", "stddev": "12.12", "min": "0000", "max": "GP", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier"}, {"transformationsData": [{"feature_label": "CptModifier2", "transformation_label": "String Indexer"}], "feature": "CptModifier2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "0.73", "stddev": "6.56", "min": "0000", "max": "59", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier2"}, {"transformationsData": [{"feature_label": "pdiagnosis1", "transformation_label": "String Indexer"}], "feature": "pdiagnosis1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "B35.4", "max": "Z01.818", "missing": "0", "distinct": "54"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis1"}, {"transformationsData": [{"feature_label": "pdiagnosis2", "transformation_label": "String Indexer"}], "feature": "pdiagnosis2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "B35.0", "max": "Z99.2", "missing": "0", "distinct": "54"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "81", "mean": "63.8", "stddev": "21.67", "min": "9", "max": "91", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "81", "mean": "175.31", "stddev": "152.97", "min": "3.76", "max": "842.13", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "Cardiovascular", "max": "Supplies", "missing": "0", "distinct": "17"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "SpecialityCategory", "transformation_label": "String Indexer"}], "feature": "SpecialityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "Cardiology", "max": "Vascular Surgery", "missing": "0", "distinct": "13"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SpecialityCategory"}, {"transformationsData": [{"feature_label": "PrimaryPayerCategory", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "Blue Cross Blue Sheild Of Florida", "max": "United Healthcare", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerCategory"}, {"transformationsData": [{"feature_label": "PrimaryPayerGroup", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "BCBS", "max": "UHC", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerGroup"}, {"transformationsData": [{"feature_label": "PrimaryPayerName", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "AARP 2ndary", "max": "Wellcare MCR HMO", "missing": "0", "distinct": "15"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerName"}, {"transformationsData": [{"feature_label": "PrimaryPayerSubGroup1", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerSubGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "Medicaid", "max": "United Healthcare Medicare Advantage", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerSubGroup1"}, {"transformationsData": [{"feature_label": "PrimaryPayerSubGroup2", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerSubGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "AmeriGroup", "max": "United Healthcare Medicare Advantage", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerSubGroup2"}, {"transformationsData": [{"feature_label": "SecondPayerCategory", "transformation_label": "String Indexer"}], "feature": "SecondPayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "Aetna US Healthcare", "max": "United Healthcare", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerCategory"}, {"transformationsData": [{"feature_label": "SecondPayerGroup", "transformation_label": "String Indexer"}], "feature": "SecondPayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "Aetna", "max": "UHC", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerGroup"}, {"transformationsData": [{"feature_label": "SecondPayerName", "transformation_label": "String Indexer"}], "feature": "SecondPayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "AARP 2ndary", "max": "Wellcare Staywell", "missing": "0", "distinct": "17"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerName"}, {"transformationsData": [{"feature_label": "SecondPayerSubGroup1", "transformation_label": "String Indexer"}], "feature": "SecondPayerSubGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "Medicaid", "max": "T_90", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerSubGroup1"}, {"transformationsData": [{"feature_label": "SecondPayerSubGroup2", "transformation_label": "String Indexer"}], "feature": "SecondPayerSubGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "Aetna", "max": "Wellcare/Staywell", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerSubGroup2"}, {"transformationsData": [{"feature_label": "ServiceItemName", "transformation_label": "String Indexer"}], "feature": "ServiceItemName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "223150.21", "stddev": "1088546.67", "min": "11042", "max": "J2405", "missing": "0", "distinct": "27"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceItemName"}, {"transformationsData": [{"feature_label": "ServiceItemCategory", "transformation_label": "String Indexer"}], "feature": "ServiceItemCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "CPTR OPHTH DX IMG POST SEGMT", "max": "Zinc paste band w >=3\"<5\"/yd", "missing": "0", "distinct": "24"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceItemCategory"}, {"transformationsData": [{"feature_label": "ServiceLocation", "transformation_label": "String Indexer"}], "feature": "ServiceLocation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "Inpatient Hospital", "max": "Office", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceLocation"}, {"transformationsData": [{"feature_label": "reasonType", "transformation_label": "String Indexer"}], "feature": "reasonType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "Denial", "max": "Denial", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "reasonType"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TransactionReasonCodeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "81", "mean": "1176.0", "stddev": "0.0", "min": "1176", "max": "1176", "missing": "0"}, "updatedLabel": "TransactionReasonCodeID"}, {"transformationsData": [{"feature_label": "Reasoncode", "transformation_label": "String Indexer"}], "feature": "Reasoncode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "22.0", "stddev": "0.0", "min": "22", "max": "22", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Reasoncode"}, {"transformationsData": [{"feature_label": "RemarkCode", "transformation_label": "String Indexer"}], "feature": "RemarkCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "N822", "max": "N822", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "RemarkCode"}, {"transformationsData": [{"feature_label": "ReasonCategory", "transformation_label": "String Indexer"}], "feature": "ReasonCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "COB", "max": "COB", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonCategory"}, {"transformationsData": [{"feature_label": "ReasonDescription", "transformation_label": "String Indexer"}], "feature": "ReasonDescription", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "This care may be covered by another payer per coordination of benefits.", "max": "This care may be covered by another payer per coordination of benefits.", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonDescription"}, {"transformationsData": [{"feature_label": "Remarkcode1", "transformation_label": "String Indexer"}], "feature": "Remarkcode1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "81", "mean": "", "stddev": "", "min": "CO22", "max": "PR22", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Remarkcode1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "label_resolution", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "81", "mean": "28.48", "stddev": "28.37", "min": "2", "max": "64", "missing": "0"}, "updatedLabel": "label_resolution"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Rank", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "81", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "Rank"}, {"feature": "ChargeDetailID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "32.7", "stddev": "22.49", "min": "0.0", "max": "72.0", "missing": "0"}, "updatedLabel": "ChargeDetailID_stringinde..."}, {"feature": "ReasonDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "81", "mean": "19.48", "stddev": "7.97", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "ReasonDate_dayofmonth"}, {"feature": "ReasonDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "81", "mean": "5.42", "stddev": "2.94", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "ReasonDate_month"}, {"feature": "ReasonDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "81", "mean": "2018.28", "stddev": "0.45", "min": "2018", "max": "2019", "missing": "0"}, "updatedLabel": "ReasonDate_year"}, {"feature": "CPTCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "5.35", "stddev": "7.28", "min": "0.0", "max": "26.0", "missing": "0"}, "updatedLabel": "CPTCode_stringindexer_tra..."}, {"feature": "CptModifier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "0.57", "stddev": "1.3", "min": "0.0", "max": "7.0", "missing": "0"}, "updatedLabel": "CptModifier_stringindexer..."}, {"feature": "CptModifier2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "81", "mean": "0.01", "stddev": "0.11", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "CptModifier2_stringindexe..."}, {"feature": "pdiagnosis1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "19.69", "stddev": "16.37", "min": "0.0", "max": "53.0", "missing": "0"}, "updatedLabel": "pdiagnosis1_stringindexer..."}, {"feature": "pdiagnosis2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "18.88", "stddev": "16.91", "min": "0.0", "max": "53.0", "missing": "0"}, "updatedLabel": "pdiagnosis2_stringindexer..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "81", "mean": "0.43", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "DepartmentCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "2.86", "stddev": "4.25", "min": "0.0", "max": "16.0", "missing": "0"}, "updatedLabel": "DepartmentCategory_string..."}, {"feature": "SpecialityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "3.67", "stddev": "3.45", "min": "0.0", "max": "12.0", "missing": "0"}, "updatedLabel": "SpecialityCategory_string..."}, {"feature": "PrimaryPayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "1.7", "stddev": "1.81", "min": "0.0", "max": "8.0", "missing": "0"}, "updatedLabel": "PrimaryPayerCategory_stri..."}, {"feature": "PrimaryPayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "1.44", "stddev": "1.52", "min": "0.0", "max": "7.0", "missing": "0"}, "updatedLabel": "PrimaryPayerGroup_stringi..."}, {"feature": "PrimaryPayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "3.1", "stddev": "3.36", "min": "0.0", "max": "14.0", "missing": "0"}, "updatedLabel": "PrimaryPayerName_stringin..."}, {"feature": "PrimaryPayerSubGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "0.88", "stddev": "1.11", "min": "0.0", "max": "5.0", "missing": "0"}, "updatedLabel": "PrimaryPayerSubGroup1_str..."}, {"feature": "PrimaryPayerSubGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "1.81", "stddev": "1.97", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "PrimaryPayerSubGroup2_str..."}, {"feature": "SecondPayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "1.59", "stddev": "2.04", "min": "0.0", "max": "8.0", "missing": "0"}, "updatedLabel": "SecondPayerCategory_strin..."}, {"feature": "SecondPayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "1.42", "stddev": "1.76", "min": "0.0", "max": "7.0", "missing": "0"}, "updatedLabel": "SecondPayerGroup_stringin..."}, {"feature": "SecondPayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "3.78", "stddev": "4.12", "min": "0.0", "max": "16.0", "missing": "0"}, "updatedLabel": "SecondPayerName_stringind..."}, {"feature": "SecondPayerSubGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "1.01", "stddev": "1.29", "min": "0.0", "max": "5.0", "missing": "0"}, "updatedLabel": "SecondPayerSubGroup1_stri..."}, {"feature": "SecondPayerSubGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "2.0", "stddev": "2.63", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "SecondPayerSubGroup2_stri..."}, {"feature": "ServiceItemName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "5.35", "stddev": "7.28", "min": "0.0", "max": "26.0", "missing": "0"}, "updatedLabel": "ServiceItemName_stringind..."}, {"feature": "ServiceItemCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "81", "mean": "4.0", "stddev": "6.38", "min": "0.0", "max": "23.0", "missing": "0"}, "updatedLabel": "ServiceItemCategory_strin..."}, {"feature": "ServiceLocation_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "81", "mean": "0.09", "stddev": "0.32", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "ServiceLocation_stringind..."}, {"feature": "reasonType_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "81", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "reasonType_stringindexer_..."}, {"feature": "Reasoncode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "81", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Reasoncode_stringindexer_..."}, {"feature": "RemarkCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "81", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "RemarkCode_stringindexer_..."}, {"feature": "ReasonCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "81", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "ReasonCategory_stringinde..."}, {"feature": "ReasonDescription_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "81", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "ReasonDescription_stringi..."}, {"feature": "Remarkcode1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "81", "mean": "0.22", "stddev": "0.5", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Remarkcode1_stringindexer..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run denialresolutionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["ChargeDetailID_stringindexer", "CPTCode_stringindexer", "CptModifier_stringindexer", "CptModifier2_stringindexer", "pdiagnosis1_stringindexer", "pdiagnosis2_stringindexer", "Age", "Gender_stringindexer", "Amount", "DepartmentCategory_stringindexer", "SpecialityCategory_stringindexer", "PrimaryPayerCategory_stringindexer", "PrimaryPayerGroup_stringindexer", "PrimaryPayerName_stringindexer", "PrimaryPayerSubGroup1_stringindexer", "PrimaryPayerSubGroup2_stringindexer", "SecondPayerCategory_stringindexer", "SecondPayerGroup_stringindexer", "SecondPayerName_stringindexer", "SecondPayerSubGroup1_stringindexer", "SecondPayerSubGroup2_stringindexer", "ServiceItemName_stringindexer", "ServiceItemCategory_stringindexer", "ServiceLocation_stringindexer", "reasonType_stringindexer", "TransactionReasonCodeID", "Reasoncode_stringindexer", "RemarkCode_stringindexer", "ReasonCategory_stringindexer", "ReasonDescription_stringindexer", "Remarkcode1_stringindexer", "Rank", "ReasonDate_dayofmonth", "ReasonDate_month", "ReasonDate_year"], "label_resolution")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

